In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
# MODEL = "claude-sonnet-4-20250514"
MODEL = "granite4:small-h"

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

if MODEL.startswith("claude"):
    model = ChatAnthropic(model=MODEL, anthropic_api_key=ANTHROPIC_API_KEY)
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
else:
    model = OllamaLLM(model=MODEL)
embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("hardware.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below.
If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke(
    {
        "context": pages[0].page_content,
        "question": "What is the main topic of the document?",
    }
)

In [ ]:
from  langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
        pages, 
        embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("Sprite blending")

In [ ]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question")
    }
    
    | prompt
    | model
)

chain.invoke({"question": "What is Sprite blending?"})

In [ ]:
questions = [
    "What is PicoVision?",
    "How does Sprite blending work?",
    "Are GPIO pins available on the PicoVision?",
    "What resolutions does the PicoVision support?"
]

for questions in questions:
    print(f"Question: {questions}")
    print(f"Answer:", chain.invoke({'question': questions}))
    print()